In [ ]:
import re
import sqlite3
import csv
con = sqlite3.connect('/content/PP_Product (3).db')
con.row_factory = sqlite3.Row
cur = con.cursor()

result = cur.execute ("Select PP_product.*, Independent_id as depends_on from PP_product Left join Product_Link on ID = Dependent_id")
result = result.fetchall()

convertRow = lambda row : {column: row[column] for column in row.keys()}
resultsToDict = list(map(lambda x : convertRow(x), result))

filename = '/content/[7172024 124 PM] Fortune PP BATCH A.txt'

filtered_lines = []

def normalize_text(text):
    return re.sub(r'[^a-zA-Z0-9\s]', '', text.lower())

def remove_number(text):
    return re.sub(r'^[0-9]+(.)\s', '', text.lower())


def Filtered_product(name, variant, SKU):
  def filterFunction(DBrecord):
    isProduct = DBrecord["Product"].lower() == name.lower()
    #isVariant = DBrecord["Variant"].lower() == variant.lower() if variant else True
    isSKU = DBrecord["Primary_SKU"].lower() in SKU.lower()
    return isProduct and isSKU #isVariant and isSKU
  return filterFunction;

def identify_products(Product_part):
    isolateProduct = Product_part.split("(")
    isolateName = isolateProduct[0].strip()
    splitNameVariant = isolateName.split("-")
    name = splitNameVariant[0].strip()
    allowed_products = list(map(lambda item : item["Product"].lower(), resultsToDict))
    if name in allowed_products:
      variant = splitNameVariant[1].strip() if len(splitNameVariant) > 1 else None
      SKU = isolateProduct[1].split(")")[0].strip()
      Filtered_items = list(filter(Filtered_product(name, variant, SKU), resultsToDict))
      if len(Filtered_items) > 0:
        return Filtered_items[0]
    return None


finalResult = {}

with open(filename, 'r') as file:
    lines = file.readlines()
    for line in lines:
        # normalized_line = normalize_text(line)
        if "===>" in line:
          formatted = remove_number(line)
          Parts = formatted.split ("===>")
          product = identify_products(Parts[0])
          if product:
            Measurement = re.search(r'[0-9\.]+', Parts[1])
            float_value = float(Measurement.group())
            Amount = re.search(r'[0-9\.]+', Parts[2])
            Float_Amount = float(Amount.group())
            if product['depends_on'] != None:
              getIndependent = list(filter(lambda item : item["ID"] == product['depends_on'], resultsToDict))[0]
              if getIndependent:
                convertkg = re.search(r'[0-9\.]+', product['Secondary_SKU'])
                Floatconvertkg  = float(convertkg.group()) if convertkg else 1
                TotalMeasurement = Floatconvertkg * float_value
                finalResult[getIndependent["ID"]] = {
                    "measurement": 0,
                    "amount" : 0,
                    "product" : getIndependent,
                    "unit" : "Some unit"
                } if getIndependent["ID"] not in finalResult else finalResult[getIndependent["ID"]]
                finalResult[getIndependent["ID"]]["measurement"] += TotalMeasurement
                finalResult[getIndependent["ID"]]["amount"] += Float_Amount
            else:
              finalResult[product["ID"]] = {
                  "measurement": 0,
                  "amount" : 0,
                  "product" : product,
                  "unit" : "Some unit"
              } if product["ID"] not in finalResult else finalResult[product["ID"]]
              finalResult[product["ID"]]["measurement"] += float_value
              finalResult[product["ID"]]["amount"] += Float_Amount


for product in finalResult.values():
    print(f"{product['product']['Product']} {product['product']['Variant']} {product['product']['Primary_SKU']} {product['measurement']} {product['unit']} ₦{product['amount']:.1f}")


with open('/content/output.csv', 'w', newline='') as csvfile:
  writer = csv.writer(csvfile)
  writer.writerow(["Product Name", "Quantity", "Amount"])
  for product in finalResult.values():
    name = f"{product['product']['Product']} {product['product'].get('Variant', '')}"
    name = name.replace(" None", "")

    quantity = int(product['measurement'])

    amount = f"{product['amount']:.1f}"
    amount = amount.replace("₦", "")

    writer.writerow([name, quantity, amount])


